In [1]:
import sys
sys.path.append('./fashion-mnist/utils/')
import mnist_reader

X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [3]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [4]:
from torch.utils.data import Dataset
from PIL import Image
class FashionMnist(Dataset):
    """ fashion-mnist dataset """
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        sample = Image.fromarray(self.X[idx].reshape(28,28))
        
        if self.transform:
            sample = self.transform(sample)
        return sample, self.y[idx]

In [5]:
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomSizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'test': transforms.Compose([
#         transforms.Scale(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }

data_transforms = transforms.Compose([
        transforms.Scale(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

/home/pxuab/anaconda3/lib/python3.6/site-packages/torchvision-0.2.0-py3.6.egg/torchvision/transforms/transforms.py:156: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.


In [70]:
FashionMnistDataset = {} 
len_train = int(0.8 * len(y_train))
FashionMnistDataset['train']= FashionMnist(X_train[:len_train], y_train[:len_train], data_transforms)
FashionMnistDataset['val']= FashionMnist(X_train[len_train:], y_train[len_train:], data_transforms)
FashionMnistDataset['test']= FashionMnist(X_test, y_test, data_transforms)

In [71]:
dataloaders = {x: torch.utils.data.DataLoader(FashionMnistDataset[x], batch_size=32,
                                             shuffle=False, num_workers=4)
              for x in ['train', 'val', 'test']}

In [72]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

In [73]:
model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.3)

In [74]:
use_gpu = True

In [75]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, batch_size=32):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, data in enumerate(dataloaders[phase]):
                # get the inputs
                inputs, labels = data
                
                inputs = inputs.expand(-1, 3, -1, -1)
                
                labels = labels.long()

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
                
                
                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#                 if i % 100 == 0:
#                     print("loss at batch %d is %.4f" %(i, loss.data[0]))
                

            epoch_loss = running_loss / (len(dataloaders[phase]) * batch_size)
            epoch_acc = running_corrects / (len(dataloaders[phase]) * batch_size)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [78]:
model_finetune = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=12)

Epoch 0/11
----------
train Loss: 0.2690 Acc: 0.9074
val Loss: 0.2129 Acc: 0.9259

Epoch 1/11
----------
train Loss: 0.1571 Acc: 0.9466
val Loss: 0.1805 Acc: 0.9357

Epoch 2/11
----------
train Loss: 0.1287 Acc: 0.9568
val Loss: 0.1764 Acc: 0.9369

Epoch 3/11
----------
train Loss: 0.1035 Acc: 0.9674
val Loss: 0.1707 Acc: 0.9396

Epoch 4/11
----------
train Loss: 0.0946 Acc: 0.9707
val Loss: 0.1706 Acc: 0.9401

Epoch 5/11
----------
train Loss: 0.0872 Acc: 0.9740
val Loss: 0.1689 Acc: 0.9408

Epoch 6/11
----------
train Loss: 0.0844 Acc: 0.9751
val Loss: 0.1689 Acc: 0.9407

Epoch 7/11
----------
train Loss: 0.0820 Acc: 0.9761
val Loss: 0.1688 Acc: 0.9407

Epoch 8/11
----------
train Loss: 0.0811 Acc: 0.9767
val Loss: 0.1689 Acc: 0.9407

Epoch 9/11
----------
train Loss: 0.0802 Acc: 0.9771
val Loss: 0.1690 Acc: 0.9404

Epoch 10/11
----------
train Loss: 0.0800 Acc: 0.9772
val Loss: 0.1690 Acc: 0.9401

Epoch 11/11
----------
train Loss: 0.0797 Acc: 0.9773
val Loss: 0.1691 Acc: 0.9401

Tr

In [79]:
def test(model, batch_size=32):
    phase = 'test'
    running_corrects = 0
    for i, data in enumerate(dataloaders[phase]):
        # get the inputs
        inputs, labels = data

        inputs = inputs.expand(-1, 3, -1, -1)

        labels = labels.long()

        # wrap them in Variable
        if use_gpu:
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs, labels = Variable(inputs), Variable(labels)

        # forward
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        running_corrects += torch.sum(preds == labels.data)
    total_acc = running_corrects / (len(dataloaders[phase]) * batch_size)
    return total_acc

In [80]:
print("Accuracy on test set with full tuning is %.4f" % test(model_finetune) )

Accuracy on test set with full tuning is 0.9342


In [66]:
model_conv = torchvision.models.resnet18(pretrained=True).eval()
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 10)

if use_gpu:
    model_conv = model_conv.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opoosed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.01, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=2, gamma=0.3)

In [67]:
model_last_layer = train_model(model_conv, criterion, optimizer_conv,
                         exp_lr_scheduler, num_epochs=12)

Epoch 0/11
----------
train Loss: 0.5823 Acc: 0.8011
val Loss: 0.4833 Acc: 0.8270

Epoch 1/11
----------
train Loss: 0.4932 Acc: 0.8334
val Loss: 0.4381 Acc: 0.8432

Epoch 2/11
----------
train Loss: 0.3892 Acc: 0.8614
val Loss: 0.3795 Acc: 0.8634

Epoch 3/11
----------
train Loss: 0.3793 Acc: 0.8635
val Loss: 0.3766 Acc: 0.8634

Epoch 4/11
----------
train Loss: 0.3538 Acc: 0.8730
val Loss: 0.3675 Acc: 0.8673

Epoch 5/11
----------
train Loss: 0.3518 Acc: 0.8735
val Loss: 0.3665 Acc: 0.8676

Epoch 6/11
----------
train Loss: 0.3445 Acc: 0.8756
val Loss: 0.3619 Acc: 0.8704

Epoch 7/11
----------
train Loss: 0.3439 Acc: 0.8761
val Loss: 0.3617 Acc: 0.8704

Epoch 8/11
----------
train Loss: 0.3416 Acc: 0.8772
val Loss: 0.3612 Acc: 0.8704

Epoch 9/11
----------
train Loss: 0.3414 Acc: 0.8771
val Loss: 0.3611 Acc: 0.8705

Epoch 10/11
----------
train Loss: 0.3407 Acc: 0.8776
val Loss: 0.3608 Acc: 0.8713

Epoch 11/11
----------
train Loss: 0.3407 Acc: 0.8775
val Loss: 0.3608 Acc: 0.8714

Tr

In [77]:
print("Accuracy on test set with last-layer tuning is %.4f" % test(model_last_layer) )

Accuracy on test set with last-layer tuning is 0.8617
